In [0]:
# # %pip install libpysal
# %pip install splot
# %pip install esda
# dbutils.library.restartPython()

## Gráficos
Realizamos los siguientes graficos como aporte a la solución planteada en la POC de UnalWater
1. Grafico de dispersión de los puntos
2. Mapa de cloropletas por barrio
3. Mapa de densidad de Kernel
4. Histograma de cantidad de productos vendidos
5. Boxplot de cantidad de productos vendidos por barrios
5. Histograma de productos vendidos por horas

In [0]:
# %pip install --upgrade matplotlib
# dbutils.library.restartPython()

In [0]:
# Importamos las librerías necesarias
from shapely.validation import make_valid
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from libpysal import weights
import esda
from splot.esda import lisa_cluster

In [0]:
# Paso 0: Importamos y editamos los nombres de los barrios

path_parquet_neigh = "/Workspace/Users/danielale22rojas@gmail.com//medellin-bigdata-poc/data/raw/medellin_neighborhoods.parquet"
gdf_barrios = gpd.read_parquet(path_parquet_neigh) 

gdf_barrios["geometry"] = gdf_barrios["geometry"].apply(make_valid)

# Primero corregimos el nombre
gdf_barrios.loc[gdf_barrios["NOMBRE"].isna(), "NOMBRE"] = "ARANJUEZ"

# Luego disolvemos por el nombre
gdf_barrios = gdf_barrios.dissolve(by="NOMBRE", as_index=False)

gdf_barrios["NOMBRE"] = gdf_barrios["NOMBRE"].str.replace("CORREGIMIENTO DE ", "", regex=True)
# gdf_barrios["NOMBRE"]

In [0]:
# Paso 1: Creamos una función para importar la capa oro como un geodataframe

def cargar_silver_como_gdf(nombre_tabla="poctesting.gold_events", crs="EPSG:4326"):
    """
    Carga la tabla Silver desde Spark y la convierte en un GeoDataFrame.
    """
    # Leer la tabla desde Spark
    df_gold = spark.table(nombre_tabla)
    
    # Pasar a Pandas
    pdf = df_gold.toPandas()

    # Crear el "geometry" a partir de lon y lat
    pdf["geometry"] = pdf.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    
    # Construir GeoDataFrame
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs=crs)
    
    return gdf

In [0]:
# Importando los datos desde oro
# bd es la base de datos gold importada como un GeodataFrame de pandas
bd = cargar_silver_como_gdf("poctesting.gold_events", "EPSG:4326")

### 1. Gráfico de dispersión de los puntos

In [0]:
# Crear el jointplot
plot = sbn.jointplot(
    x='longitude', 
    y='latitude', 
    data=bd, 
    s=5, 
    height=8
)

# Obtener el eje principal del jointplot
ax = plot.ax_joint

# Dibujar los límites de barrios encima
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5)

# Personalizar títulos y etiquetas
plot.fig.suptitle("Distribución de puntos de venta con límites de barrios", y=1.02, fontsize=16)
plot.set_axis_labels("Longitud", "Latitud")

plt.show()

### 2. Mapa de cloropletas por barrio

In [0]:
# crear dataframe unido
df_gold = spark.table("poctesting.gold_events")
pdf_gold = df_gold.select("district", "avg_by_neighborhood", "total_by_neighborhood").distinct().toPandas()

# Unir los dos dataframes
gdf_merged = gdf_barrios.merge(pdf_gold, left_on="NOMBRE", right_on="district", how="left")

In [0]:

# Grafico coroplético
f, ax = plt.subplots(1, figsize=(12,7))
gdf_merged.plot(
    ax=ax,
    column="avg_by_neighborhood",  # usamos el total ya calculado
    legend=True,
    scheme="Quantiles",
    legend_kwds={"fmt": "{:.0f}"},
    cmap="Blues",
    edgecolor="black",
    linewidth=0.5
)

# Dibujar límites de barrios
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5, alpha=0.5)

# Añadir nombres de barrios
for idx, row in gdf_barrios.iterrows():
    centroid = row.geometry.centroid
    ax.text(
        centroid.x, centroid.y, 
        str(row["NOMBRE"]), 
        fontsize=6, color="black", ha="center"
    )

ax.set_axis_off()
ax.set_title("Promedio de productos vendidos por barrio", fontsize=14)
plt.axis("equal")
plt.show()

### 3. Mapa de densidad de Kernel

In [0]:
from matplotlib.cm import ScalarMappable

# Crear figura
f, ax = plt.subplots(figsize=(10, 12))

# KDE con seaborn
sns_plot = sbn.kdeplot(
    x=bd["longitude"], 
    y=bd["latitude"], 
    fill=True, 
    cmap="viridis_r", 
    levels=40, 
    alpha=0.7, 
    ax=ax
)

# Dibujar límites de barrios
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5, alpha=0.5)

# Añadir nombres de barrios
for idx, row in gdf_barrios.iterrows():
    centroid = row.geometry.centroid
    ax.text(
        centroid.x, centroid.y, 
        str(row["NOMBRE"]), 
        fontsize=7, color="black", ha="center"
    )

# Ajustar límites al bounding box de Medellín
bounds = gdf_barrios.total_bounds
ax.set_xlim(bounds[0], bounds[2])
ax.set_ylim(bounds[1], bounds[3])

# Crear colorbar manual (gradiente)
sm = ScalarMappable(cmap="viridis_r")
sm.set_array([])  # necesario para inicializar
cbar = f.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.set_label("Densidad estimada de eventos", fontsize=12)

# Estilo final
ax.set_title("Mapa de densidad de eventos de ventas en Medellín", fontsize=16, pad=20)
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [0]:
import folium
from folium.plugins import HeatMap

# Calcular centro aproximado de Medellín
lat_center = bd["latitude"].mean()
lon_center = bd["longitude"].mean()

# Crear mapa centrado en Medellín
m = folium.Map(location=[lat_center, lon_center], zoom_start=12, tiles="OpenStreetMap")

# Añadir Heatmap con tus puntos
HeatMap(
    data=bd[["latitude", "longitude"]].values, 
    radius=15,       # tamaño del radio de cada punto
    blur=25,         # suavizado
    max_zoom=12,     # nivel de zoom máximo para ajustar densidad
    min_opacity=0.4  # opacidad mínima
).add_to(m)

# Añadir polígonos de barrios
folium.GeoJson(
    gdf_barrios.to_json(),
    name="Límites de barrios",
    style_function=lambda feature: {
        "color": "black", 
        "weight": 0.5, 
        "fillOpacity": 0
    },
    tooltip=folium.GeoJsonTooltip(fields=["NOMBRE"], aliases=["Barrio"])
).add_to(m)

# Control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa en Databricks
m



### 4. Histograma de cantidad de productos vendidos

In [0]:
# Histograma
plt.figure(figsize=(10,6))
plt.hist(bd["quantity_products"], bins=30, color="skyblue", edgecolor="black")
plt.title("Histograma de productos vendidos - Poisson No homogenea", fontsize=16)
plt.xlabel("Cantidad de productos")
plt.ylabel("Frecuencia")
plt.grid(alpha=0.3)
plt.show()

In [0]:
# Boxplot
# Calcular medianas de productos vendidos por barrio
orden = bd.groupby("district")["quantity_products"].median().sort_values().index

# Crear el boxplot con orden
plt.figure(figsize=(14,7))
sbn.boxplot(data=bd, x="district", y="quantity_products", order=orden)
plt.title("Boxplot de productos vendidos por barrio (ordenado por mediana)", fontsize=16)
plt.xlabel("Barrio")
plt.ylabel("Cantidad de productos")
plt.xticks(rotation=90)
plt.show()

### 5. Dependencia Espacial y Clustering

In [0]:
# Creamos la matriz de pesos espacial tipo queen
w_queen = weights.Queen.from_dataframe(gdf_merged)
w = weights.Queen.from_dataframe(bd)

In [0]:
# Grafico de moran con matriz de pesos queen

from splot.esda import plot_moran

# plot_moran(esda.Moran(gdf_merged['avg_by_neighborhood'], w_queen));

In [0]:
from splot.esda import lisa_cluster

# Ahora hacemos el grafico de lisa
lisa = esda.Moran_Local(gdf_merged['avg_by_neighborhood'], w_queen)
# lisa_cluster(lisa, gdf_merged);

In [0]:
from splot.esda import plot_local_autocorrelation
plot_local_autocorrelation(lisa, gdf_merged, 'avg_by_neighborhood');

In [0]:
from splot.esda import plot_moran

# mi = esda.Moran(bd['quantity_products'], w)
# print(mi.I)
# # mi.p_sim
# plot_moran(mi);

In [0]:
lisa = esda.Moran_Local(bd['quantity_products'], w)
from splot.esda import lisa_cluster
# lisa_cluster(lisa, bd);

In [0]:
from splot.esda import plot_local_autocorrelation
plot_local_autocorrelation(lisa, bd, 'quantity_products');

### Culstering

In [0]:
# CLUSTERING CON DBSCAN
# from sklearn.cluster import DBSCAN
# import numpy as np

# # Extraer coordenadas
# coords = np.array(list(zip(bd.geometry.x, bd.geometry.y)))

# # DBSCAN con distancia en coordenadas
# db = DBSCAN(eps=0.01, min_samples=5).fit(coords)  
# bd["cluster"] = db.labels_

# # Revisar resultados
# print(bd["cluster"].value_counts())

# # Graficar
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(10,8))
# bd.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
# ax.set_title("Clustering de eventos (DBSCAN)", fontsize=14)
# plt.show()

In [0]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Seleccionar variables para clustering
X = bd[["longitude", "latitude", "quantity_products"]].dropna()

# Escalar variables (recomendado)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calcular KMeans para distintos valores de k
inertias = []
K = range(1, 11)  # probamos k entre 1 y 10
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8,5))
plt.plot(K, inertias, "o-", linewidth=2)
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del codo para determinar k óptimo")
plt.grid(True)
plt.show()


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Seleccionar variables: coordenadas + cantidad de productos
X = bd[["longitude", "latitude", "quantity_products"]].dropna()

# Escalar variables (muy importante porque están en magnitudes distintas)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ajustar KMeans
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
bd["cluster"] = kmeans.fit_predict(X_scaled)

# Graficar clusters en el mapa
fig, ax = plt.subplots(figsize=(10, 8))
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5, alpha=0.5)
bd.plot(
    ax=ax, column="cluster", categorical=True, legend=True, 
    markersize=10, cmap="tab20"
)
ax.set_title("Clustering de eventos (K-Means con cantidad de productos)", fontsize=14)
plt.show()

# Resumen: cantidad media de productos por cluster
print(bd.groupby("cluster")["quantity_products"].mean())


In [0]:
bd.columns

In [0]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# calcular el centroide de los poligonos 
gdf_merged["centroid"] = gdf_merged.geometry.centroid

# Ver coordenadas de los centroides
gdf_merged["centroid_lon"] = gdf_merged.centroid.x
gdf_merged["centroid_lat"] = gdf_merged.centroid.y

# Variables de clustering
X = gdf_merged[["avg_by_neighborhood", "centroid_lon", "centroid_lat"]].dropna()

# Escalar las variables (muy importante)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calcular inercia para distintos valores de k
inertias = []
K = range(1, 11)  # probamos k de 1 a 10
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8,5))
plt.plot(K, inertias, "o-", linewidth=2)
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia (Within-Cluster Sum of Squares)")
plt.title("Método del Codo para KMeans")
plt.grid(True)
plt.show()


In [0]:
from sklearn import cluster
kmeans5 = cluster.KMeans(n_clusters=3)

# Ajustar el modelo
k5cls = kmeans5.fit(gdf_merged[['avg_by_neighborhood', 'centroid_lon', 'centroid_lat']]);

In [0]:
gdf_merged['k5cls'] = k5cls.labels_

# Setup figure and ax
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot unique values choropleth including a legend and with no boundary lines
gdf_merged.plot(column='k5cls', categorical=True, legend=True, linewidth=0, ax=ax)
# Remove axis
ax.set_axis_off()
# Add title
plt.title('Ventas por barrio Kmeans, K=4')
# Display the map
plt.show()

In [0]:

# Volver a Spark
neighbourhood_gold = spark.createDataFrame(gdf_merged.drop(columns=["geometry", "centroid"]))

# Guardamos los dataframes como tablas en delta para los dashboards
# 1) Guardar como tabla delta en el metastore
(
  neighbourhood_gold.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("poctesting.neighbourhood_gold")
)


In [0]:
# Volver a Spark
points_gold = spark.createDataFrame(bd.drop(columns="geometry"))

# Guardamos los dataframes como tablas en delta para los dashboards
# 1) Guardar como tabla delta en el metastore
(
  points_gold.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("poctesting.points_gold")
)